In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
from time import sleep
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('whitegrid')
import re

## Scraper

Create driver

In [ ]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--window-size=1920x1080')
driver = webdriver.Chrome(options=chrome_options)


Go to oddschecker website

In [ ]:
url = 'https://www.oddschecker.com/football/football-specials'
driver.get(url)

Sometimes there is annoying popup

In [ ]:
try:
    driver.find_element_by_xpath('//*[@id="promo-modal"]/div[1]/div/div/div[5]/a').click()
except:
    print('no popup')
sleep(1)

Find the transfer rumours


In [ ]:
fixtures = driver.find_element_by_xpath('//*[@id="outrights"]/div/ul')
markets = fixtures.find_elements_by_tag_name('li')
links = [m.find_element_by_tag_name('a').get_attribute('href') for m in markets]
transfer_links = [l for l in links if  'club-after-summer-transfer-window' in l]

Helper method for parsing odds string (including fractions)

In [ ]:
def parse_odds(x):
    x_str = str(x)
    if '/' in x_str:
        num, denom = x_str.split('/')
        return int(num) / int(denom)
    else:
        try:
            return int(x)
        except:
            return np.nan

Scrape all transfer rumours

In [ ]:
combined_df = pd.DataFrame()

In [ ]:
for i, l in enumerate(transfer_links):
    print(l, end="")
    print()
    print(f'{i}/{len(transfer_links)}')
    print()
    sleep(1)
    driver.get(l)
    tables = pd.read_html(driver.page_source)
    # Odds table seems to be last table on page
    odds_df = tables[-1]
    # Remove crazy long column names
    odds_df.columns = [''] * len(odds_df.columns)
    # Transpose - clubs along axis
    clean_df = odds_df.T.rename(columns=odds_df.T.iloc[0])
    # Calculate lowest odds - most likely
    long_df = pd.DataFrame(clean_df.applymap(parse_odds).min()).reset_index()
    # Add in column names, including player name
    long_df.columns = ['destination', 'odds']
    long_df['player'] = l.split('/')[-2].replace('-',' ').title()
    # Append to df
    combined_df = combined_df.append(long_df,sort=False)

In [ ]:
combined_df['probability'] = 1 / (1 + combined_df['odds'])

## Plotting

plotting functions, by club and by player

In [ ]:
def get_player_destinations(df,player):
    club_df = df.loc[(df.player==player) &
                             (df.probability > 0)].sort_values(by='probability',ascending=False)
    
    plt.subplots(figsize=(20,15))
    ax = sns.barplot(data=club_df,y='destination',x='probability',orient='h')
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=90)
    ax.set_title(f'{player} Destinations \n',{'fontsize':20})
    plt.savefig(f'output/players/{player}.png')

In [ ]:
def get_club_targets(df,club):
    club_df = df.loc[(df.destination==club) &
                             (df.probability > 0)].sort_values(by='probability',ascending=False)
    plt.subplots(figsize=(20,15))
    ax = sns.barplot(data=club_df,y='player',x='probability',orient='h')
    ax.set_title(f'{club} Transfer Targets \n',{'fontsize':20})
    plt.savefig(f'output/clubs/{club}.png')

In [ ]:
for club in ['Arsenal','Chelsea','Real Madrid', 'PSG', 'Barcelona','Man City','Man Utd','West Ham','Juventus']:
    get_club_targets(combined_df,club)

In [ ]:
for player in ['Eden Hazard','Adrian Rabiot','David De Gea', 'Matthijs De Ligt','Gareth Bale', 'Kylian Mbappe','Koulibaly Kalidou','Neymar',
              'Jadon Sancho','Alexis Sanchez']:
    get_player_destinations(combined_df,player)

In [ ]:
most_likely = combined_df.loc[~combined_df.destination.str.contains('To Stay|To Leave|Any')].sort_values(by='probability',ascending=False).head(30)
most_likely['transfer'] = most_likely.player + ' - ' + most_likely['destination']
plt.subplots(figsize=(20,15))
ax = sns.barplot(data=most_likely,y='transfer',x='probability',orient='h')
ax.set_title('30 most likely Transfers \n',{'fontsize':20})
plt.savefig(f'output/most likely overall.png')